In [ ]:
#
#O conjunto de dados analisados é sobre frutas
#https://www.kaggle.com/moltean/fruits

#Apenas algumas frutas do dataset foram usadas: 
#-Grape
#-Orange
#-Nectarine
#-Mandarine
#-Banana
#-Kiwi
#-Apple


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # opencv
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score

from keras import preprocessing
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

In [12]:
img = cv2.imread('./fruits-360/Test/Apple/3_100.jpg')

In [13]:
img.shape

(100, 100, 3)

In [14]:
input_dir = './fruits-360/'

In [113]:
#tipo = Training ou Test
classes = {'Apple':0,'Banana':1, 'Grape': 2, 'Kiwi':3, 'Mandarine':4, 'Nectarine':5, 'Orange':6}
def ler_dataset(tipo):
  X = []
  y = []
  qtd = 1000 if tipo == 'Training' else 200
  #para cada pasta
  for pasta in os.listdir(input_dir+tipo+'/'):
    print(pasta)
    #para cada imagem na pasta
    for img_file in os.listdir(input_dir+tipo+'/'+pasta+'/')[:qtd]:
      #leia a imagem
      img = cv2.imread(input_dir+tipo+'/'+pasta+'/'+img_file)
      #transforme imagem em array de pixels e salve na lista de imagens
      X.append(img)
      #guarde a classificação da imagem
      y.append(classes[pasta])
  X = np.asarray(X)
  y = np.asarray(y)
  return X,y

In [114]:
#Treino
X_treino,y_treino = ler_dataset('Training')

Grape
Orange
Nectarine
Mandarine
Banana
Kiwi
Apple


In [115]:
#Visualizando o shape X_treino
X_treino.shape

(3399, 100, 100, 3)

In [116]:
#Visualizando o shape do y_treino
y_treino.shape

(3399,)

In [117]:
# Test
X_teste,y_teste = ler_dataset('Test')

Grape
Orange
Nectarine
Mandarine
Banana
Kiwi
Apple


In [118]:
#Visualizando o shape X_test
X_teste.shape

(1142, 100, 100, 3)

In [119]:
#Visualizando o shape Y_test
y_teste.shape

(1142,)

In [120]:
#Visualizando a quantidade de classes
no_of_classes = len(np.unique(y_treino))
no_of_classes

7

In [121]:
#transformando labels de numérico para categórico
y_treino = to_categorical(y_treino, num_classes = 7)
y_teste = to_categorical(y_teste, num_classes = 7)

In [123]:
X_treino = X_treino/255
X_teste = X_teste/255

In [124]:

#definição do modelo
model = Sequential()

# 2 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (100, 100, 3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 128 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

#  2 camadas fully conected 
model.add(Flatten())#converte os mapas de ativação finais para um veter de 1 dimensão
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(7, activation = "softmax"))

In [125]:
# Definindo o algoritmo otmizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [126]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [127]:
# Descrição do modelo
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_99 (Conv2D)           (None, 100, 100, 32)      2432      
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 100, 100, 32)      25632     
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 100, 100, 32)      25632     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_44 (Dropout)         (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 50, 50, 64)        18496     
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 50, 50, 64)      

In [128]:
# treinamento do modelo - o valor da acurácia foi baixo provalvemente pela quantidade de imagens que foram utilizadas 
model.fit(X_treino, y_treino, epochs=1,verbose = 1)

107/107 [==============================] - 305s 3s/step - loss: 1.9480 - accuracy: 0.1392


In [129]:
model.evaluate(X_teste,y_teste)

36/36 [==============================] - 21s 592ms/step - loss: 1.9458 - accuracy: 0.1436


[1.945834755897522, 0.1436077058315277]